# Tilts and q's with temp using Carsten's COM:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import lib.calc as calc
import h5py
from imp import reload
import scipy

### Prepare data

In [ ]:
slist =  np.load('../stitched_data/scan_list.npy')
slist

In [ ]:
num = 0
fname = str(slist[num][0]) + '_' + str(slist[num][3])
gint = calc.read_h5_stitched_e16014(fname,'../E16014_stitch.h5') # gridded stitched intensity
Qx,Qy,Qz = calc.read_q_h5_stitched_e16014(fname,'../E16014_stitch.h5') # Q's
qx, qy, qz = calc.make_qmesh(Qx, Qy, Qz, gint) # mesh the q's

Define the ROI and inspect

In [ ]:
# iroi = np.s_[230:300,250:300,140:200]
# iroi = np.s_[280:400,240:300,50:150]
# iroi = np.s_[230:290,250:300,100:170]

Quick inspection:

In [ ]:
if num <= 5:
    iroi = np.s_[280:400,240:300,50:150]
else:
    iroi = np.s_[230:290,250:300,100:170]
roi, qxr, qyr, qzr = [array[iroi] for array in (gint, qx, qy, qz)]
roi += 1
print(roi.shape)

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(11,4))
ax1.pcolormesh(qxr[:,:,0],qyr[:,:,0],roi.sum(axis=2),cmap='jet')
ax1.set_ylabel('Qy');ax1.set_xlabel('Qx')
ax1.set_aspect('equal')

ax2.pcolormesh(qxr[:,0,:],qzr[:,0,:],roi.sum(axis=1),cmap='jet')
ax2.set_ylabel('Qz');ax2.set_xlabel('Qx')
ax2.set_aspect('equal')

ax3.pcolormesh(qyr[0,:,:],qzr[0,:,:],roi.sum(axis=0),cmap='jet')
ax3.set_ylabel('Qz');ax3.set_xlabel('Qy')
ax3.set_aspect('equal')

plt.show()

### Calculate COR

In [ ]:
take = int(30)
# print(take/roi.size*100)
isort = roi.ravel().argsort()[::-1][:take]
roisort = roi.ravel()[isort]  - roi.ravel()[isort].min()
qCOM = [(arr.ravel()[isort]*roisort).sum() / roisort.sum() for arr in (qxr,qyr,qzr)]

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(11,4))
fig.suptitle(fname)

ax1.pcolormesh(qxr[:,:,0],qyr[:,:,0],roi.sum(axis=2),cmap='jet')
ax1.set_ylabel('Qy');ax1.set_xlabel('Qx')
ax1.scatter(qCOM[0],qCOM[1],c='b')
ax1.set_aspect('equal')

ax2.pcolormesh(qxr[:,0,:],qzr[:,0,:],roi.sum(axis=1),cmap='jet')
ax2.set_ylabel('Qz');ax2.set_xlabel('Qx')
ax2.scatter(qCOM[0],qCOM[2],c='b')
ax2.set_aspect('equal')

ax3.pcolormesh(qyr[0,:,:],qzr[0,:,:],roi.sum(axis=0),cmap='jet')
ax3.set_ylabel('Qz');ax3.set_xlabel('Qy')
ax3.scatter(qCOM[1],qCOM[2],c='b')
ax3.set_aspect('equal')

plt.savefig('croi_'+fname)
plt.show()
print(qCOM)

### Calculate $|\vec{q}|$ and tilts